In [ ]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# New Section

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train_v1.csv')
test = pd.read_csv('/content/drive/MyDrive/test_v1.csv')

In [ ]:
features = list(train.columns)
target = 'Anomaly'
features.remove(target)
features.remove('Index')
features

['Station',
 'Lag1',
 'Lag2',
 'Lag5',
 'Lag10',
 'Lag30',
 'Lag50',
 'diff1',
 'diff2',
 'diff5',
 'diff30',
 'diff50',
 'RollingMean_2',
 'RollingSD_2',
 'RollingMean_3',
 'RollingSD_3',
 'RollingMean_4',
 'RollingSD_4',
 'RollingMean_5',
 'RollingSD_5',
 'RollingMean_25',
 'RollingSD_25',
 'RollingMean_35',
 'RollingSD_35',
 'RollingMean_50',
 'RollingSD_50',
 'RollingMean_100',
 'RollingSD_100',
 'RollingMean_200',
 'RollingSD_200',
 'RollingMean_400',
 'RollingSD_400',
 'RollingMean_800',
 'RollingSD_800',
 'meanaddsd',
 'meandiffsd']

In [ ]:
dtrain = xgb.DMatrix(train[features], label=train[target])
dtest = xgb.DMatrix(test[features], label=test[target])

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 6,
    'learning_rate': 0.1,
    'seed': 42,
    'tree_method': 'gpu_hist'
}




In [ ]:
bst = xgb.train(params, dtrain, num_boost_round=100)

In [ ]:
y_pred = bst.predict(dtest)
y_pred_labels = np.round(y_pred)


In [ ]:
precision = precision_score(test[target], y_pred_labels,zero_division = 0)
recall = recall_score(test[target], y_pred_labels)
f1 = f1_score(test[target], y_pred_labels)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.5986196424874043
Recall: 0.6816245697310721
F1 Score: 0.6374312843578211


In [ ]:

xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42,
    tree_method = 'gpu_hist'
)

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [9,12,15],
    'learning_rate': [0.02, 0.04],
    'subsample': [0.6, 1.0],
    'colsample_bytree': [0.6, 1.0],
    'gamma': [0.1, 0.2]
}


In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
tscv = TimeSeriesSplit(n_splits=5)
grid_search = GridSearchCV(
    xgb_clf,
    param_grid,
    scoring='f1',
    n_jobs=-1,
    cv=tscv,
    verbose=1
)

In [ ]:
grid_search.fit(train[features], train[target])

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/usr/local/lib/python3.9/dist-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     im...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=-1,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 1.0], 'gamma': [0.1, 0.2],
                         'learning_rate': [0.02, 0.04],
                         'max_depth': [9, 12, 15],
                         'n_estimators': [50, 100, 150],
                         'subsample': [0.6, 1.0]},
             scoring='f1', verbose=1)

In [ ]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_


In [ ]:
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [ ]:
best_params

{'colsample_bytree': 1.0,
 'gamma': 0.1,
 'learning_rate': 0.02,
 'max_depth': 12,
 'n_estimators': 50,
 'subsample': 0.6}

In [ ]:
test_predictions = best_model.predict(test[features])

#y_pred = bst.predict(dtest)
#y_pred_labels = np.round(y_pred)
precision = precision_score(test[target], test_predictions,zero_division = 0)
recall = recall_score(test[target], test_predictions)
f1 = f1_score(test[target], test_predictions)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.6219464711956798
Recall: 0.5774484070226176
F1 Score: 0.598871990480382


In [ ]:

xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42,
    tree_method = 'gpu_hist',
    max_depth = 12,
    gamma = 1
)
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.0005, 0.0008,0.001],
    'subsample': [0.05,0.1]
}
tscv = TimeSeriesSplit(n_splits=3)
grid_search = GridSearchCV(
    xgb_clf,
    param_grid,
    scoring='f1',
    n_jobs=-1,
    cv=tscv,
    verbose=1
)
grid_search.fit(train[features], train[target])


Fitting 3 folds for each of 18 candidates, totalling 54 fits


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=1, gpu_id=None,
                                     grow_policy=None, impor...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=12,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=-1,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.0005, 0.0008, 0.001],
                         'n_estimators': [50, 100, 150],
                         'subsample': [0.05, 0.1]},
             scoring='f1', verbose=1)

In [ ]:
best_2model = grid_search.best_estimator_
best_2params = grid_search.best_params_
test_2predictions = best_2model.predict(test[features])

#y_pred = bst.predict(dtest)
#y_pred_labels = np.round(y_pred)
precision = precision_score(test[target], test_2predictions,zero_division = 0)
recall = recall_score(test[target], test_2predictions)
f1 = f1_score(test[target], test_2predictions)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.7152134032501583
Recall: 0.6924382691793848
F1 Score: 0.7036415908001915


In [ ]:
best_2params

{'learning_rate': 0.001, 'n_estimators': 50, 'subsample': 0.1}

In [ ]:
xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    n_jobs=-1,
    random_state=42,
    tree_method = 'gpu_hist',
    learning_rate = 0.001,
    n_estimators = 50,
    subsample = 0.1,
    gamma = 1
)
param_grid = {
    'max_depth' : [5,8,10,12,15,18]
}
tscv = TimeSeriesSplit(n_splits=3)
grid_search = GridSearchCV(
    xgb_clf,
    param_grid,
    scoring='f1',
    n_jobs=-1,
    cv=tscv,
    verbose=1
)
grid_search.fit(train[features], train[target])


Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=1, gpu_id=None,
                                     grow_policy=None, impor...
                                     learning_rate=0.001, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=50, n_jobs=-1,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=42, ...),
             n_jobs=-1, param_grid={'max_depth': [5, 8, 10, 12, 15, 18]},
             scoring='f1', verbose=1)

In [ ]:
best_3model = grid_search.best_estimator_
best_3params = grid_search.best_params_
test_3predictions = best_3model.predict(test[features])

#y_pred = bst.predict(dtest)
#y_pred_labels = np.round(y_pred)
precision = precision_score(test[target], test_3predictions,zero_division = 0)
recall = recall_score(test[target], test_3predictions)
f1 = f1_score(test[target], test_3predictions)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.7158327816135439
Recall: 0.7137513163478616
F1 Score: 0.7147905336806731
